In [1]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage
from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraries
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
#import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [3]:
#Import Data
nfl_data = nfl.import_schedules([2023])
original_df = nfl_data.groupby('week')

In [4]:
# Select relevant columns for team vs. team DataFrame
team_vs_team_df = nfl_data[['week', 'away_team', 'away_score', 'home_team', 'home_score']]


In [5]:
# Get data from games
weekly_nfl = nfl.import_weekly_data([2023])
weekly_nfl.head()

Downcasting floats.


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NYJ,2023,1,REG,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,4,REG,...,0.0,0.483465,0,0.0,0.031250,-0.012397,0.038197,0.0,0.8,1.8
2,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,7,REG,...,1.0,1.437224,0,3.2,0.034483,0.094340,0.117762,0.0,1.6,2.6
3,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,11,REG,...,0.0,-0.547367,0,0.0,0.045455,0.036885,0.094001,0.0,0.0,0.0
4,00-0024243,M.Lewis,Marcedes Lewis,TE,TE,https://static.www.nfl.com/image/private/f_aut...,CHI,2023,14,REG,...,0.0,-0.190257,0,0.0,0.033333,-0.012987,0.040909,0.0,0.4,1.4


In [6]:
#clean up data to show data columns for each game
columns = [ 'week','recent_team','opponent_team','passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa'
           ]


In [7]:
#create Df
games_df = weekly_nfl[columns]
games_df.head()

,week,recent_team,opponent_team,passing_epa,passing_yards,rushing_epa,rushing_yards,receiving_epa
0,1,NYJ,BUF,-2.03196,0.0,NaN,0.0,NaN
1,4,CHI,DEN,NaN,0.0,NaN,0.0,0.483465
2,7,CHI,LV,NaN,0.0,NaN,0.0,1.437224
3,11,CHI,DET,NaN,0.0,NaN,0.0,-0.547367
4,14,CHI,DET,NaN,0.0,NaN,0.0,-0.190257


In [8]:
# Groupby team and week offense stats
team_by_week = games_df.groupby(['week', 'recent_team'])
team_by_week.head()
team_stats_per_game = team_by_week.sum()
team_stats_per_game.index.set_names(['week', 'team_offense'], inplace=True)
team_stats_per_game = team_stats_per_game.reset_index()
#team_stats_per_game_off = team_stats_per_game.drop(columns='index')

team_stats_per_game_off = team_stats_per_game.rename(columns={'team_defense': 'team_offense'})
# List of columns to add '_def' to
columns_to_suffix = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

team_stats_per_game_off = team_stats_per_game_off.rename(columns={col: col + '_offense' for col in columns_to_suffix})

team_stats_per_game_off.tail()

,week,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
565,21,DET,6.998413,273.0,2.268643,182.0,10.135550
566,21,KC,7.783350,241.0,-8.881944,89.0,12.629683
567,21,SF,6.624600,267.0,7.201090,155.0,9.344954
568,22,KC,6.061147,333.0,-7.526628,130.0,13.647686
569,22,SF,9.144471,276.0,-9.394331,110.0,12.717166


In [9]:
# Groupby team and week defense stats
team_by_week_def = games_df.groupby(['week', 'opponent_team'])
team_by_week_def.head()
team_stats_per_game_def = team_by_week_def.sum()

# List of columns to add '_def' to
columns_to_suffix_def = ['passing_epa',
           'passing_yards', 'rushing_epa','rushing_yards','receiving_epa']

# Add '_def' suffix to the specified columns
team_stats_per_game_def = team_stats_per_game_def.rename(columns={col: col + '_def' for col in columns_to_suffix_def})
team_stats_per_game_def.index.set_names(['week', 'team_defense'], inplace=True)
team_stats_per_game_def = team_stats_per_game_def.reset_index()


In [10]:
team_stats_per_game_def.tail()

,week,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
565,21,DET,6.624600,267.0,7.201090,155.0,9.344954
566,21,KC,-5.030445,272.0,2.078561,81.0,0.962597
567,21,SF,6.998413,273.0,2.268643,182.0,10.135550
568,22,KC,9.144471,276.0,-9.394331,110.0,12.717166
569,22,SF,6.061147,333.0,-7.526628,130.0,13.647686


In [11]:
# merge both dfs so it shows offense and defense stats per game
team_total_df = team_stats_per_game.merge(team_stats_per_game_def, left_on='team_offense', right_on='team_defense')
# Filter rows where 'team_offense' is equal to 'team_defense'
team_total_df = team_total_df[team_total_df['team_offense'] == team_total_df['team_defense']]
#remove duplicates
team_total_df = team_total_df[(team_total_df['team_offense'] == team_total_df['team_defense']) & (team_total_df['week_x'] == team_total_df['week_y'])]
team_total_df.head(1)

,week_x,team_offense,passing_epa,passing_yards,rushing_epa,rushing_yards,receiving_epa,week_y,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.79934,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773


In [12]:
# create a copy of team_total_df for home team and away team

# Make the first copy of the original DataFrame
home_team_df = team_total_df.copy()

# Make a second copy from the first copy
away_team_df = home_team_df.copy()

In [13]:
#make a home_team df
# Add '_home' to each column name
for column in home_team_df.columns:
    new_column_name_home = column + '_home'
    home_team_df.rename(columns={column: new_column_name_home}, inplace=True)
    
home_team_df.head()

,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,rushing_yards_home,receiving_epa_home,week_y_home,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.799340,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773
18,2,ARI,7.430885,228.0,2.366217,151.0,7.653436,2,ARI,8.180348,321.0,6.719742,127.0,11.507128
36,3,ARI,4.560631,189.0,7.001422,222.0,8.408302,3,ARI,-3.659615,249.0,8.314601,185.0,0.522374
54,4,ARI,3.898311,265.0,5.478536,105.0,6.421503,4,ARI,22.844091,283.0,2.675849,124.0,24.983665
72,5,ARI,-15.651534,166.0,-1.152939,142.0,-5.404606,5,ARI,9.498447,317.0,-8.893337,93.0,16.416182


In [14]:
#make a away_team df
# Add '_away' to each column name
for column in away_team_df.columns:
    new_column_name_away = column + '_away'
    away_team_df.rename(columns={column: new_column_name_away}, inplace=True)
away_team_df.head()

,week_x_away,team_offense_away,passing_epa_away,passing_yards_away,rushing_epa_away,rushing_yards_away,receiving_epa_away,week_y_away,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.799340,1,ARI,-13.413063,202.0,-5.708867,92.0,1.409773
18,2,ARI,7.430885,228.0,2.366217,151.0,7.653436,2,ARI,8.180348,321.0,6.719742,127.0,11.507128
36,3,ARI,4.560631,189.0,7.001422,222.0,8.408302,3,ARI,-3.659615,249.0,8.314601,185.0,0.522374
54,4,ARI,3.898311,265.0,5.478536,105.0,6.421503,4,ARI,22.844091,283.0,2.675849,124.0,24.983665
72,5,ARI,-15.651534,166.0,-1.152939,142.0,-5.404606,5,ARI,9.498447,317.0,-8.893337,93.0,16.416182


In [15]:
# Make df with points and stats

# merge home team with home stats
home_team = team_vs_team_df.merge(home_team_df, left_on='home_team', right_on='team_offense_home')
#remove duplicates
home_team = home_team[(home_team['home_team'] ==  home_team['team_offense_home']) & (home_team['week_x_home'] == home_team['week'])]
home_team.tail(10)

,week,away_team,away_score,home_team,home_score,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,rushing_yards_home,receiving_epa_home,week_y_home,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home
4944,18,BUF,21.0,MIA,14.0,18,MIA,-0.179888,173.0,2.018006,108.0,1.253841,18,MIA,3.935818,359.0,1.620589,128.0,11.626987
4948,3,PIT,23.0,LV,18.0,3,LV,-8.352474,324.0,-2.516379,69.0,-2.013242,3,LV,5.921596,235.0,-6.737266,105.0,9.555780
4967,5,GB,13.0,LV,17.0,5,LV,-1.803250,208.0,-3.381476,96.0,6.604667,5,LV,-14.767113,182.0,4.087527,110.0,-11.481108
4985,6,NE,17.0,LV,21.0,6,LV,9.882565,264.0,-10.972410,84.0,11.559125,6,LV,-4.471819,200.0,4.164759,83.0,3.130720
5005,9,NYG,6.0,LV,30.0,9,LV,7.603049,209.0,-7.482466,125.0,9.152975,9,LV,-19.027122,200.0,-4.605570,123.0,-7.132212
5023,10,NYJ,12.0,LV,16.0,10,LV,-9.807395,153.0,-1.637336,148.0,-3.269739,10,LV,-4.556659,263.0,-3.730515,108.0,0.062029
5042,12,KC,31.0,LV,17.0,12,LV,3.499284,248.0,-5.031726,123.0,6.922385,12,LV,12.587396,298.0,-0.940888,69.0,15.450614
5060,14,MIN,3.0,LV,0.0,14,LV,-19.203550,171.0,-0.846225,56.0,-15.269022,14,LV,-12.121390,146.0,-2.298960,132.0,0.565894
5078,15,LAC,21.0,LV,63.0,15,LV,12.323879,260.0,3.581412,124.0,16.015894,15,LV,-10.320502,257.0,-13.295756,92.0,7.879056
5098,18,DEN,14.0,LV,27.0,18,LV,9.306783,244.0,-1.255764,129.0,17.544756,18,LV,0.353189,272.0,-2.951732,48.0,9.411932


In [16]:
# merge away team with home stats
away_team = home_team.merge(away_team_df, left_on='away_team', right_on='team_offense_away')
#remove duplicates
away_team = away_team[(away_team['away_team'] ==  away_team['team_offense_away']) & (away_team['week_x_away'] == away_team['week'])]
away_team.head(10)

,week,away_team,away_score,home_team,home_score,week_x_home,team_offense_home,passing_epa_home,passing_yards_home,rushing_epa_home,...,rushing_epa_away,rushing_yards_away,receiving_epa_away,week_y_away,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away
0,1,DET,21.0,KC,20.0,1,KC,-6.453613,226.0,-2.581925,...,-3.636585,118.0,6.054341,1,DET,-6.453613,226.0,-2.581925,90.0,-5.052831
26,7,DET,6.0,BAL,38.0,7,BAL,24.334782,357.0,1.219639,...,1.834399,84.0,0.542692,7,DET,24.334782,357.0,1.219639,146.0,24.334782
54,16,DET,30.0,MIN,24.0,16,MIN,5.691321,411.0,-4.351473,...,-0.709332,143.0,16.405670,16,DET,5.691321,411.0,-4.351473,17.0,13.313128
71,13,DET,33.0,NO,28.0,13,NO,6.018938,267.0,-8.409609,...,-1.068709,142.0,10.316429,13,DET,6.018938,267.0,-8.409609,113.0,10.585057
92,14,DET,13.0,CHI,28.0,14,CHI,2.945851,223.0,-0.283723,...,-2.966777,140.0,0.374628,14,DET,2.945851,223.0,-0.283723,142.0,10.205959
108,10,DET,41.0,LAC,38.0,10,LAC,14.587980,323.0,-0.869600,...,11.664154,200.0,19.774326,10,DET,14.587980,323.0,-0.869600,98.0,14.587980
125,6,DET,20.0,TB,6.0,6,TB,-8.300112,206.0,-4.832714,...,-7.840065,40.0,22.679295,6,DET,-8.300112,206.0,-4.832714,46.0,-7.366295
155,17,DET,19.0,DAL,20.0,17,DAL,11.299153,345.0,-7.875702,...,-5.493361,125.0,5.870264,17,DET,11.299153,345.0,-7.875702,61.0,7.770870
179,21,DET,31.0,SF,34.0,21,SF,6.624600,267.0,7.201090,...,2.268643,182.0,10.135550,21,DET,6.624600,267.0,7.201090,155.0,9.344954
183,4,DET,34.0,GB,20.0,4,GB,-12.693180,246.0,0.423209,...,4.333835,211.0,6.932714,4,DET,-12.693180,246.0,0.423209,27.0,-5.993364


In [17]:
# rename df
full_df = away_team.copy()

In [18]:
#full_df.to_excel('training_nfl.xlsx', index=False)


In [19]:
# Create Columns for training
column_train_x = [
   'passing_epa_home','passing_yards_home','rushing_epa_home','rushing_yards_home','receiving_epa_home',
    'passing_epa_def_home','passing_yards_def_home','rushing_epa_def_home','rushing_yards_def_home','receiving_epa_def_home',
       'passing_epa_away','passing_yards_away','rushing_epa_away','rushing_yards_away','receiving_epa_away',
           'passing_epa_def_away','passing_yards_def_away','rushing_epa_def_away','rushing_yards_def_away','receiving_epa_def_away',


]




column_train_y = ['home_score','away_score']

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# model
# Split data into features (X) and target variable (y)
X = full_df[column_train_x]
y = full_df[column_train_y]

# Encode categorical variables using one-hot encoding if needed

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")


Mean Absolute Error (MAE): 4.25987992788616
Mean Squared Error (MSE): 27.654206553979552
Root Mean Squared Error (RMSE): 5.248728600356156
R-squared (R2): 0.7536782661853927


In [21]:
team_stats_per_game_off.head()

,week,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,1,ARI,-9.239668,132.0,-11.179296,96.0,-2.799340
1,1,ATL,-4.450450,115.0,-0.595772,130.0,0.718799
2,1,BAL,-4.415663,169.0,0.155152,110.0,7.985488
3,1,BUF,-1.409397,236.0,-3.168568,97.0,5.100840
4,1,CAR,-10.528563,146.0,-8.897445,154.0,-3.103000


In [22]:
### Create mean of these stats to create test data
#home team
mean_offense = team_stats_per_game_off.groupby('team_offense').mean()
mean_offense = mean_offense.drop('week', axis=1)
mean_offense = mean_offense.reset_index()

mean_offense.head()

,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,ARI,-5.599671,201.764709,1.079086,139.117645,-0.294376
1,ATL,-2.596495,222.058823,-3.726649,127.000000,1.929744
2,BAL,3.194072,226.578949,0.983369,156.368423,8.006257
3,BUF,5.263197,247.105270,2.552425,135.421051,7.869932
4,CAR,-9.277600,190.882355,-2.079348,104.058823,0.631979


In [23]:
#defense team
# Split the DataFrame into non-numerical and numerical columns
mean_defense = team_stats_per_game_def.groupby('team_defense').mean()
# Remove the week column
mean_defense = mean_defense.drop('week', axis=1)
mean_defense = mean_defense.reset_index()

mean_defense.head()


,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def
0,ARI,4.779762,226.235291,0.159482,143.176468,8.724047
1,ATL,-0.100534,218.058823,-4.040809,118.176468,5.339218
2,BAL,-6.485339,217.526321,-2.295621,104.578949,0.364174
3,BUF,-1.511967,216.842102,-0.611747,112.210526,2.616188
4,CAR,0.321117,182.882355,-0.171240,122.411766,3.704572


In [24]:
# Merge on defense vs offense per team
test_data_df = pd.merge(mean_defense, mean_offense, left_on='team_defense',right_on='team_offense', how='left')
test_data_df.head()

,team_defense,passing_epa_def,passing_yards_def,rushing_epa_def,rushing_yards_def,receiving_epa_def,team_offense,passing_epa_offense,passing_yards_offense,rushing_epa_offense,rushing_yards_offense,receiving_epa_offense
0,ARI,4.779762,226.235291,0.159482,143.176468,8.724047,ARI,-5.599671,201.764709,1.079086,139.117645,-0.294376
1,ATL,-0.100534,218.058823,-4.040809,118.176468,5.339218,ATL,-2.596495,222.058823,-3.726649,127.000000,1.929744
2,BAL,-6.485339,217.526321,-2.295621,104.578949,0.364174,BAL,3.194072,226.578949,0.983369,156.368423,8.006257
3,BUF,-1.511967,216.842102,-0.611747,112.210526,2.616188,BUF,5.263197,247.105270,2.552425,135.421051,7.869932
4,CAR,0.321117,182.882355,-0.171240,122.411766,3.704572,CAR,-9.277600,190.882355,-2.079348,104.058823,0.631979


In [25]:
# create DF for upcoming week
# create stats to show upcpoming games

#2024 update to get 2024 schedule
nfl_data_two = nfl.import_schedules([2024])

grouped = nfl_data_two.groupby('week')
# Select the group corresponding to week 6
week_data = grouped.get_group(5)
prior_week = grouped.get_group(1)
get_teams =['home_team','away_team','home_score','away_score']
schedule_week_df = week_data[get_teams]
schedule_week_df.reset_index(drop=True, inplace=True)
schedule_week_df.head()

,home_team,away_team,home_score,away_score
0,ATL,TB,NaN,NaN
1,MIN,NYJ,NaN,NaN
2,CHI,CAR,NaN,NaN
3,CIN,BAL,NaN,NaN
4,HOU,BUF,NaN,NaN


In [26]:
# create a copy of team_total_df for home team and away team for testing

# Make the first copy of the original DataFrame
home_team_test = test_data_df.copy()

# Make a second copy from the first copy
away_team_test = home_team_test.copy()

In [27]:
# Add '_home' to each column name
for column in home_team_test.columns:
    new_column_name_test = column + '_home'
    home_team_test.rename(columns={column: new_column_name_test}, inplace=True)
    
home_team_test.head()

,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,team_offense_home,passing_epa_offense_home,passing_yards_offense_home,rushing_epa_offense_home,rushing_yards_offense_home,receiving_epa_offense_home
0,ARI,4.779762,226.235291,0.159482,143.176468,8.724047,ARI,-5.599671,201.764709,1.079086,139.117645,-0.294376
1,ATL,-0.100534,218.058823,-4.040809,118.176468,5.339218,ATL,-2.596495,222.058823,-3.726649,127.000000,1.929744
2,BAL,-6.485339,217.526321,-2.295621,104.578949,0.364174,BAL,3.194072,226.578949,0.983369,156.368423,8.006257
3,BUF,-1.511967,216.842102,-0.611747,112.210526,2.616188,BUF,5.263197,247.105270,2.552425,135.421051,7.869932
4,CAR,0.321117,182.882355,-0.171240,122.411766,3.704572,CAR,-9.277600,190.882355,-2.079348,104.058823,0.631979


In [28]:
# Add '_away' to each column name
for column in away_team_test.columns:
    new_column_name_test_away = column + '_away'
    away_team_test.rename(columns={column: new_column_name_test_away}, inplace=True)
    
away_team_test.head()

,team_defense_away,passing_epa_def_away,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away,team_offense_away,passing_epa_offense_away,passing_yards_offense_away,rushing_epa_offense_away,rushing_yards_offense_away,receiving_epa_offense_away
0,ARI,4.779762,226.235291,0.159482,143.176468,8.724047,ARI,-5.599671,201.764709,1.079086,139.117645,-0.294376
1,ATL,-0.100534,218.058823,-4.040809,118.176468,5.339218,ATL,-2.596495,222.058823,-3.726649,127.000000,1.929744
2,BAL,-6.485339,217.526321,-2.295621,104.578949,0.364174,BAL,3.194072,226.578949,0.983369,156.368423,8.006257
3,BUF,-1.511967,216.842102,-0.611747,112.210526,2.616188,BUF,5.263197,247.105270,2.552425,135.421051,7.869932
4,CAR,0.321117,182.882355,-0.171240,122.411766,3.704572,CAR,-9.277600,190.882355,-2.079348,104.058823,0.631979


In [29]:
# Make df with points and stats for testing upcoming week

# merge home team with home stats
testing_home = schedule_week_df.merge(home_team_test, left_on='home_team', right_on='team_offense_home')
#remove duplicates
#testing_home = home_team[(home_team['home_team'] ==  home_team['team_offense_home']) & (home_team['week_x_home'] == home_team['week'])]
testing_home.head(1)

,home_team,away_team,home_score,away_score,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,team_offense_home,passing_epa_offense_home,passing_yards_offense_home,rushing_epa_offense_home,rushing_yards_offense_home,receiving_epa_offense_home
0,ATL,TB,NaN,NaN,ATL,-0.100534,218.058823,-4.040809,118.176468,5.339218,ATL,-2.596495,222.058823,-3.726649,127.0,1.929744


In [30]:
# merge away team with home stats
testing_away = testing_home.merge(away_team_test, left_on='away_team', right_on='team_offense_away')
#remove duplicates
#away_team = away_team[(away_team['away_team'] ==  away_team['team_offense_away']) & (away_team['week_x_away'] == away_team['week'])]
testing_away.head(10)

,home_team,away_team,home_score,away_score,team_defense_home,passing_epa_def_home,passing_yards_def_home,rushing_epa_def_home,rushing_yards_def_home,receiving_epa_def_home,...,passing_yards_def_away,rushing_epa_def_away,rushing_yards_def_away,receiving_epa_def_away,team_offense_away,passing_epa_offense_away,passing_yards_offense_away,rushing_epa_offense_away,rushing_yards_offense_away,receiving_epa_offense_away
0,ATL,TB,NaN,NaN,ATL,-0.100534,218.058823,-4.040809,118.176468,5.339218,...,268.684204,-2.641788,92.947365,7.082324,TB,3.389954,248.947372,-4.026836,90.368423,7.904936
1,MIN,NYJ,NaN,NaN,MIN,1.289153,252.235291,-2.683147,98.705879,7.164492,...,187.176468,-3.321178,124.000000,0.887091,NYJ,-10.980999,198.411758,-4.241757,96.882355,-2.224875
2,CHI,CAR,NaN,NaN,CHI,0.731251,249.705887,-3.495640,86.352943,4.044536,...,182.882355,-0.171240,122.411766,3.704572,CAR,-9.277600,190.882355,-2.079348,104.058823,0.631979
3,CIN,BAL,NaN,NaN,CIN,1.744545,263.529419,-0.041415,126.176468,7.166279,...,217.526321,-2.295621,104.578949,0.364174,BAL,3.194072,226.578949,0.983369,156.368423,8.006257
4,HOU,BUF,NaN,NaN,HOU,0.251718,252.263153,-2.763757,101.473686,6.118454,...,216.842102,-0.611747,112.210526,2.616188,BUF,5.263197,247.105270,2.552425,135.421051,7.869932
5,JAX,IND,NaN,NaN,JAX,-0.884788,256.176483,-1.049457,103.058823,3.491396,...,244.529419,-1.083767,123.823532,5.592357,IND,-0.068466,228.352936,-2.416049,121.117645,5.270685
6,NE,MIA,NaN,NaN,NE,0.480952,223.000000,-5.484513,93.176468,5.426205,...,244.111115,-1.417274,99.833336,6.416550,MIA,5.532857,272.055542,-0.593957,132.444443,8.645572
7,WAS,CLE,NaN,NaN,WAS,6.300512,272.176483,-0.835399,126.764709,10.828694,...,190.500000,-2.895814,103.833336,-0.560389,CLE,-6.823284,239.888885,-3.054811,115.166664,-1.176837
8,DEN,LV,NaN,NaN,DEN,2.393801,253.058823,-1.158291,137.117645,8.072318,...,229.411758,-2.424559,118.529411,4.194424,LV,-2.647270,215.647064,-3.308651,90.705879,2.970538
9,SF,ARI,NaN,NaN,SF,-2.718376,237.449997,-0.788657,98.650002,2.582541,...,226.235291,0.159482,143.176468,8.724047,ARI,-5.599671,201.764709,1.079086,139.117645,-0.294376


In [31]:
# rename df
full_test = testing_away.copy()
#full_test = full_test.reset_index()

full_test = full_test.rename(columns={
    'passing_epa_offense_away': 'passing_epa_away',
    'passing_yards_offense_away': 'passing_yards_away',
    'receiving_epa_offense_away': 'receiving_epa_away',
    'rushing_epa_offense_away': 'rushing_epa_away',
    'rushing_yards_offense_away': 'rushing_yards_away',
     'passing_epa_offense_home': 'passing_epa_home',
    'passing_yards_offense_home': 'passing_yards_home',
    'receiving_epa_offense_home': 'receiving_epa_home',
    'rushing_epa_offense_home': 'rushing_epa_home',
    'rushing_yards_offense_home': 'rushing_yards_home',
})
full_test.dtypes

home_team                  object
away_team                  object
home_score                float64
away_score                float64
team_defense_home          object
passing_epa_def_home      float32
passing_yards_def_home    float32
rushing_epa_def_home      float32
rushing_yards_def_home    float32
receiving_epa_def_home    float32
team_offense_home          object
passing_epa_home          float32
passing_yards_home        float32
rushing_epa_home          float32
rushing_yards_home        float32
receiving_epa_home        float32
team_defense_away          object
passing_epa_def_away      float32
passing_yards_def_away    float32
rushing_epa_def_away      float32
rushing_yards_def_away    float32
receiving_epa_def_away    float32
team_offense_away          object
passing_epa_away          float32
passing_yards_away        float32
rushing_epa_away          float32
rushing_yards_away        float32
receiving_epa_away        float32
dtype: object

In [32]:
column_test_x = ['passing_epa_def_away', 'passing_yards_def_away',
       'rushing_epa_def_away', 'rushing_yards_def_away',
       'receiving_epa_def_away','passing_epa_away', 'passing_yards_away',
    'rushing_epa_away', 'rushing_yards_away','receiving_epa_away',
     'passing_epa_def_home', 'passing_yards_def_home',
       'rushing_epa_def_home', 'rushing_yards_def_home',
       'receiving_epa_def_home','passing_epa_home', 'passing_yards_home',
    'rushing_epa_home', 'rushing_yards_home',
       'receiving_epa_home',            
                ]

In [33]:
# Now, use the trained model to predict score for df_test
X_test = full_test[column_test_x]
y_pred = model.predict(X_test)

# The predictions are stored in y_pred
print("Predicted Scores:")
print(y_pred)

Predicted Scores:
[[21.520359 20.789997]
 [22.442709 17.372658]
 [22.822222 15.960297]
 [20.292656 24.175293]
 [21.544292 22.584553]
 [23.319513 20.870857]
 [18.863308 21.79119 ]
 [19.85904  21.660553]
 [21.554476 20.127787]
 [29.15224  17.524668]
 [24.266634 21.850187]
 [24.13189  19.49146 ]
 [20.16811  22.741915]
 [22.50977  19.612411]]


/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [34]:
# add predicted scores into df: 
predict_cols = ['home_team','away_team']
df_predictions = full_test[predict_cols]
scores_df = pd.DataFrame(y_pred, columns=['home_score', 'away_score'])
df_predictions = pd.concat([df_predictions, scores_df], axis=1)
df_predictions['Over/Under'] = df_predictions['home_score'] + df_predictions['away_score']
# Assuming 'home_score', 'away_score', 'home_team', and 'away_team' are columns in your DataFrame
df_predictions['Favorite'] = np.where(df_predictions['home_score'] > df_predictions['away_score'], df_predictions['home_team'], df_predictions['away_team'])
df_predictions['Spread'] = df_predictions['away_score'] - df_predictions['home_score']
df_predictions['spread'] = df_predictions['Spread'].apply(lambda x: '-' + str(x) if x >= 0 else str(x))
df_predictions = df_predictions.drop('Spread', axis=1)

df_predictions.head(17)

,home_team,away_team,home_score,away_score,Over/Under,Favorite,spread
0,ATL,TB,21.520359,20.789997,42.310356,ATL,-0.7303619384765625
1,MIN,NYJ,22.442709,17.372658,39.815369,MIN,-5.070051193237305
2,CHI,CAR,22.822222,15.960297,38.782516,CHI,-6.86192512512207
3,CIN,BAL,20.292656,24.175293,44.467949,BAL,-3.8826370239257812
4,HOU,BUF,21.544292,22.584553,44.128845,BUF,-1.0402603149414062
5,JAX,IND,23.319513,20.870857,44.190369,JAX,-2.4486560821533203
6,NE,MIA,18.863308,21.791189,40.654495,MIA,-2.9278812408447266
7,WAS,CLE,19.859039,21.660553,41.519592,CLE,-1.801513671875
8,DEN,LV,21.554476,20.127787,41.682262,DEN,-1.4266891479492188
9,SF,ARI,29.152241,17.524668,46.676910,SF,-11.627573013305664


In [35]:
# Convert the 'spread' column to float
df_predictions['spread'] = pd.to_numeric(df_predictions['spread'], errors='coerce')
# Round the numeric columns to 1 decimal place
df_predictions['home_score'] = df_predictions['home_score'].round(2)
df_predictions['away_score'] = df_predictions['away_score'].round(2)
df_predictions['Over/Under'] = df_predictions['Over/Under'].round(2)
df_predictions['spread'] = df_predictions['spread'].round(2)
# Set display precision
pd.set_option('display.float_format', '{:.1f}'.format)



df_predictions.head(16)

,home_team,away_team,home_score,away_score,Over/Under,Favorite,spread
0,ATL,TB,21.5,20.8,42.3,ATL,-0.7
1,MIN,NYJ,22.4,17.4,39.8,MIN,-5.1
2,CHI,CAR,22.8,16.0,38.8,CHI,-6.9
3,CIN,BAL,20.3,24.2,44.5,BAL,-3.9
4,HOU,BUF,21.5,22.6,44.1,BUF,-1.0
5,JAX,IND,23.3,20.9,44.2,JAX,-2.5
6,NE,MIA,18.9,21.8,40.7,MIA,-2.9
7,WAS,CLE,19.9,21.7,41.5,CLE,-1.8
8,DEN,LV,21.5,20.1,41.7,DEN,-1.4
9,SF,ARI,29.1,17.5,46.7,SF,-11.6


In [36]:
#df_predictions.to_excel('week_1_pred.xlsx', index=False)


In [37]:
# Add in actual odds from these games
vegas_df = pd.read_excel('vegas_odds/week_one_vegas_odds.xlsx')
vegas_df.head(16)


,home_team,away_team,Over/Under_Vegas,Favorite_Vegas,Spread_Vegas
0,KC,BAL,46.0,KC,-3.0
1,PHI,GB,48.5,PHI,-2.5
2,CIN,NE,40.5,CIN,-8.5
3,CHI,TEN,45.0,CHI,-3.5
4,MIA,JAX,49.5,MIA,-3.0
5,IND,HOU,49.0,HOU,-2.5
6,BUF,ARI,48.0,BUF,-6.0
7,ATL,PIT,42.0,ATL,-3.0
8,NYG,MIN,41.5,MIN,-1.5
9,NO,CAR,41.5,NO,-4.0


In [38]:
#merge so model and vegas odds are on each game
# Merge DataFrames
model_vegas_df = pd.merge(df_predictions, vegas_df, on=['home_team', 'away_team'])
model_vegas_df['Over/Under_Difference'] = model_vegas_df['Over/Under'] - model_vegas_df['Over/Under_Vegas']


model_vegas_df.head(16)


,home_team,away_team,home_score,away_score,Over/Under,Favorite,spread,Over/Under_Vegas,Favorite_Vegas,Spread_Vegas,Over/Under_Difference


In [147]:
from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def display_dataframe():
    return df_predictions.to_html()

if __name__ == '__main__':
    app.run()










 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Sep/2024 15:44:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2024 15:44:54] "GET /favicon.ico HTTP/1.1" 404 -
